In [2]:
import zipfile
import os

with zipfile.ZipFile(os.path.join(os.getcwd(),'cifar10.zip'),'r') as zip_ref:
    zip_ref.extractall()

In [3]:
import torch
from torch import nn

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import numpy as np
from PIL import  Image
import pathlib

In [4]:
train_dir = os.path.join(os.getcwd(),'cifar10','train')
test_dir  = os.path.join(os.getcwd(),'cifar10','test')


In [5]:
class_names = [name for name in os.listdir(test_dir)]
class_names.sort()
print(class_names)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [6]:
class_dict = {name:num for num,name in enumerate(class_names)}
print(class_dict)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [7]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
class cifarDS(Dataset):

    def __init__(self,path:str,transform:transforms = None):
        self.path = path
        self.transform = transform
        self.all_images = list(pathlib.Path(self.path).glob('*/*.png'))


    def __len__(self):
        return len(self.all_images)
    
    def getImage(self,path:str) -> Image.Image: 

        return Image.open(path)

    def __getitem__(self, index:str ) -> tuple[torch.Tensor,int]:

        path = self.all_images[index]

        image = self.getImage(path)
        classdir = os.path.basename(os.path.dirname(path))
        classname = class_dict[classdir]

        # y_onehot = torch.nn.functional.one_hot(torch.tensor(classname), num_classes=10).float()

        if self.transform:
            return self.transform(image) ,classname
        

        else:
            self.transform = transforms.Compose([
                transforms.ToTensor()
            ])

            return self.transform(image), classname





        

In [9]:
import matplotlib.pyplot as plt
def visualize_image(img_tensor):
    img  = img_tensor.permute(1,2,0)

    plt.imshow(img)
    plt.show()


In [10]:
train_dataset = cifarDS(path=train_dir)
print(train_dataset[40000])



(tensor([[[0.9725, 0.9647, 0.9765,  ..., 0.9412, 0.9686, 0.9843],
         [0.9529, 0.9176, 0.8667,  ..., 0.4471, 0.4941, 0.7451],
         [0.6549, 0.6157, 0.5843,  ..., 0.4000, 0.3451, 0.4784],
         ...,
         [0.4392, 0.4510, 0.4745,  ..., 0.3176, 0.3176, 0.3176],
         [0.4588, 0.4706, 0.4784,  ..., 0.3059, 0.3059, 0.3098],
         [0.4824, 0.4667, 0.4784,  ..., 0.3020, 0.3098, 0.3059]],

        [[0.9961, 0.9922, 1.0000,  ..., 0.9569, 0.9843, 1.0000],
         [1.0000, 0.9608, 0.9098,  ..., 0.4941, 0.5373, 0.7843],
         [0.7059, 0.6667, 0.6353,  ..., 0.4510, 0.4000, 0.5333],
         ...,
         [0.4980, 0.5098, 0.5333,  ..., 0.3843, 0.3922, 0.3882],
         [0.5059, 0.5176, 0.5216,  ..., 0.3725, 0.3765, 0.3804],
         [0.5216, 0.5059, 0.5137,  ..., 0.3725, 0.3804, 0.3765]],

        [[1.0000, 1.0000, 1.0000,  ..., 0.9569, 0.9804, 0.9961],
         [1.0000, 0.9843, 0.9333,  ..., 0.5176, 0.5569, 0.8039],
         [0.7412, 0.7020, 0.6745,  ..., 0.4980, 0.4431, 0

In [11]:
ImageTransform = transforms.Compose([
    transforms.Resize((224,224)),

    transforms.ToTensor()
])

In [12]:
trainDataset = cifarDS(train_dir,ImageTransform)
testDataset = cifarDS(test_dir,ImageTransform)


In [13]:
trainDataLoader = DataLoader(trainDataset,  batch_size=2,shuffle=True)
testDataLoader  = DataLoader(testDataset,batch_size =1,shuffle=False)

In [14]:
test_iterator = iter(testDataLoader)
print(next(test_iterator))  

[tensor([[[[0.0824, 0.0824, 0.0824,  ..., 0.0824, 0.0824, 0.0824],
          [0.0824, 0.0824, 0.0824,  ..., 0.0824, 0.0824, 0.0824],
          [0.0824, 0.0824, 0.0824,  ..., 0.0824, 0.0824, 0.0824],
          ...,
          [0.3098, 0.3098, 0.3098,  ..., 0.1686, 0.1686, 0.1686],
          [0.3098, 0.3098, 0.3098,  ..., 0.1686, 0.1686, 0.1686],
          [0.3098, 0.3098, 0.3098,  ..., 0.1686, 0.1686, 0.1686]],

         [[0.0314, 0.0314, 0.0314,  ..., 0.0431, 0.0431, 0.0431],
          [0.0314, 0.0314, 0.0314,  ..., 0.0431, 0.0431, 0.0431],
          [0.0314, 0.0314, 0.0314,  ..., 0.0431, 0.0431, 0.0431],
          ...,
          [0.3137, 0.3137, 0.3137,  ..., 0.1961, 0.1961, 0.1961],
          [0.3137, 0.3137, 0.3137,  ..., 0.1961, 0.1961, 0.1961],
          [0.3137, 0.3137, 0.3137,  ..., 0.1961, 0.1961, 0.1961]],

         [[0.0902, 0.0902, 0.0902,  ..., 0.0863, 0.0863, 0.0863],
          [0.0902, 0.0902, 0.0902,  ..., 0.0863, 0.0863, 0.0863],
          [0.0902, 0.0902, 0.0902,  ..., 

# building model

In [15]:
class AlexNet(nn.Module):
    

    def __init__(self):
        super().__init__()

        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=96,kernel_size=11,stride=4,),
                                    nn.ReLU(),
                                    nn.LocalResponseNorm(size=5,alpha =0.001,beta=0.75,k=1),
                                    nn.MaxPool2d(3,2)
        )

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5),
                                    nn.ReLU(),
                                    nn.LocalResponseNorm(5,0.001,0.75,1),
                                    nn.MaxPool2d(3,2))

        self.layer3 = nn.Sequential(nn.Conv2d(in_channels= 256,out_channels=384,kernel_size=3),
                                    nn.ReLU()
        )

        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=384,out_channels=384,kernel_size=3),
                                    nn.ReLU())

        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=384,out_channels=256,kernel_size=3),
                                    nn.ReLU(),
                                    nn.Flatten())
            

        self.fc = nn.Sequential(nn.Dropout(),
                                nn.Linear(in_features=4096,out_features=4096),
                                nn.ReLU(),
                                nn.Linear(in_features=4096,out_features=4096),
                                nn.ReLU(),
                                nn.Linear(in_features=4096,out_features=10)
                                )
  
    def forward(self,x):

        return (self.fc(self.layer5(self.layer4(self.layer3(self.layer2(self.layer1(x)))))))
    




In [16]:
alexnet = AlexNet()
alexnet.to(device)

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=1)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): LocalResponseNorm(5, alpha=0.001, beta=0.75, k=1)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4096, out_features=4096, bias=True)
    (2): ReLU()
 

In [17]:
optim = torch.optim.Adam(alexnet.parameters(),)
criteration = nn.CrossEntropyLoss()

In [18]:
from tqdm import tqdm


In [21]:
iterna = iter(trainDataLoader)
print(next(iterna))

[tensor([[[[0.7725, 0.7725, 0.7725,  ..., 0.7490, 0.7490, 0.7490],
          [0.7725, 0.7725, 0.7725,  ..., 0.7490, 0.7490, 0.7490],
          [0.7725, 0.7725, 0.7725,  ..., 0.7490, 0.7490, 0.7490],
          ...,
          [0.4078, 0.4078, 0.4078,  ..., 0.2039, 0.2039, 0.2039],
          [0.4078, 0.4078, 0.4078,  ..., 0.2039, 0.2039, 0.2039],
          [0.4078, 0.4078, 0.4078,  ..., 0.2039, 0.2039, 0.2039]],

         [[0.7725, 0.7725, 0.7725,  ..., 0.7529, 0.7529, 0.7529],
          [0.7725, 0.7725, 0.7725,  ..., 0.7529, 0.7529, 0.7529],
          [0.7725, 0.7725, 0.7725,  ..., 0.7529, 0.7529, 0.7529],
          ...,
          [0.4588, 0.4588, 0.4588,  ..., 0.1843, 0.1843, 0.1843],
          [0.4588, 0.4588, 0.4588,  ..., 0.1843, 0.1843, 0.1843],
          [0.4588, 0.4588, 0.4588,  ..., 0.1843, 0.1843, 0.1843]],

         [[0.7725, 0.7725, 0.7725,  ..., 0.7725, 0.7725, 0.7725],
          [0.7725, 0.7725, 0.7725,  ..., 0.7725, 0.7725, 0.7725],
          [0.7725, 0.7725, 0.7725,  ..., 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [28]:
print(np.arange(1,10,1))

[1 2 3 4 5 6 7 8 9]


In [31]:
def plot_predictions(trainLoss, testLoss, epoch):
    epoch = np.arange(0,epoch+1,1)
    plt.plot(trainLoss,epoch,label="train loss")
    plt.plot(testLoss,epoch, label='test loss ')
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.legend()
    plt.show()

In [25]:
trainDataLoader.dataset

In [33]:
epochs = 1000
trainLosslist = []
testLosslist =[]
trainLoss = 0
testLoss = 0


for i in tqdm(range(epochs)):
    alexnet.train()
    for x, y in trainDataLoader:
        x,y = x.to(device),y.to(device)
        
        optim.zero_grad()
        train_pred = alexnet(x)
        loss = criteration(train_pred,y)

        loss.backward()

        optim.step()

        trainLoss += loss
        
    alexnet.eval()
    with torch.inference_mode():
        for test_x,test_y in testDataLoader:
            test_x,test_y = x.to(device), y.to(device)
            test_pred = alexnet(x)
            test_loss = criteration(test_pred,y)

            testLoss += loss

    print(f'Train Loss after iteration {i} : {trainLoss/len(trainDataLoader.dataset)}')
    print(f'Test Loss occured after iteration {i} : {testLoss/len(testDataLoader.dataset)}')
    

    trainLosslist.append(trainLoss/len(trainDataLoader.dataset))
    testLosslist.append(testLoss/len(testDataLoader.dataset))

    plot_predictions(trainLosslist.to('cpu'), testLosslist.to('cpu'), i)
            


  0%|          | 0/1000 [06:48<?, ?it/s]

Train Loss after iteration 0 : 1.1515791416168213
Test Loss occured after iteration 0 : 2.3228166103363037


AttributeError: 'list' object has no attribute 'to'